# TP de datos personales

## infovis, 1c2024

Alumno: Pedro Lopez Guzman

In [ ]:
!pip install altair
!pip install pandas

In [ ]:
!pip install pyvis

In [ ]:
!wget "https://raw.githubusercontent.com/Pedro-UwU/infovis/main/TP1/data/owned_games_with_tags.json"

In [ ]:
import altair as alt
import pandas as pd
import json
from pprint import pprint

games = pd.read_json('owned_games_with_tags.json')

In [ ]:
# Tiempo total de juego por juego

chart = alt.Chart(games).transform_calculate(
    hours = "datum.playtime_forever / 60"
).encode(
    x = alt.X("hours:Q", title="Tiempo (hours)"),
    y = alt.Y("name", sort ='-x', title="Juego"),
    text = alt.Text("hours:Q", format=".2f"),
    tooltip=alt.Tooltip("hours:Q", format=".2f")
).transform_window(
    rank = 'rank(playtime_forever)',
    sort = [alt.SortField('playtime_forever', order='descending')]
).transform_filter(
    (alt.datum.rank < 20)
)
chart = chart.mark_bar() + chart.mark_text(align='left', dx=10)
chart.configure_axis(
    grid=False
).configure_view(
    stroke=None
)

In [ ]:
# Porcentaje de juegos que jugué vs no jugué

pie_chart = alt.Chart(games).transform_calculate(
    playtime_cat = "datum.playtime_forever > 0 ? 'Jugado' : 'Nunca Jugado'"
).transform_aggregate(
    count="count()",
    groupby=["playtime_cat"]
).encode(
    alt.Theta("count:Q"),
    alt.Color("playtime_cat:N", legend=alt.Legend(title="He jugado?")),
    tooltip=['playtime_category:N', 'count:Q']
)

pie = pie_chart.mark_arc(radius=200)
pie.configure_axis(
    grid=False
).configure_view(
    stroke=None
)

In [ ]:
# Crear un nuevo dataframe con las categorías
game_tags_list = list()
for _, row in games.iterrows():
  game_name = row['name']
  game_playtime = row['playtime_forever']
  tags = row['tags']
  if len(tags) == 0:
    continue
  for t in tags:
    game_tags_list.append({"Name": game_name, "Playtime": game_playtime, "tag": t})

games_tags_df = pd.DataFrame(game_tags_list)
print(games_tags_df.head())

In [ ]:
# Graficar las tags mas comunes
common_tags = alt.Chart(games_tags_df).transform_aggregate(
    Count="count()",
    groupby=["tag"]
).encode(
    x=alt.X("Count:Q"),
    y=alt.Y("tag:N", sort='-x'),
    tooltip="Count:Q"
).transform_window(
    rank="rank(Count:Q)",
    sort=[alt.SortField("Count:Q", order="descending")]
).transform_filter(
    (alt.datum.Count > 20)
)

common_tags.mark_bar()

In [ ]:
# Grafico de red usando pyvis
from pyvis.network import Network


cum_tags = games_tags_df.groupby("tag")['Playtime'].sum().reset_index()
cum_games = games_tags_df.groupby("Name")['Playtime'].sum().reset_index()
cum_tags = cum_tags[cum_tags['Playtime'] > 0]
cum_games = cum_games[cum_games['Playtime'] > 0]

net = Network(width="1000px")

for _, game in cum_games.iterrows():
  net.add_node(game['Name'], title=game['Name'])
